In [2]:
!pip install neo4j-driver
!pip install graphdatascience

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.0/224.0 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j-driver: filename=neo4j_driver-5.27.0-py3-none-any.whl size=301902 sha256=4e3d5b15c1ee43e02e159f25b086c86b8ccb3b953362b03366c94042bd769283
  Stored in directory: /root/.cache/pip/wheels/97/9c/63/9b5a51b678fce6807d92992ffa75382bacc8df2a5016efc986
Successfully built neo4j-driver
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.7/301.7 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 18.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 17.0.0
    Uninstalling pyarrow-17.0.0:
      Successfully uninstalled pyarrow-17.0.0


graphdatascince  interacting with database and running cypher quries


In [3]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
from graphdatascience import GraphDataScience


In [ ]:
from google.colab import userdata


# Retrieve secrets securely using userdata.get()
NEO4J_URI = userdata.get('NEO4J_URI')
NEO4J_USERNAME = userdata.get('NEO4J_USERNAME')
NEO4J_PASSWORD = userdata.get('NEO4J_PASSWORD')

# Check if secrets are retrieved successfully
if not all([NEO4J_URI, NEO4J_USERNAME, NEO4J_PASSWORD]):
    raise ValueError("One or more secrets could not be retrieved. Please check your Colab secrets setup.")

# Connect to the Graph Data Science library
gds = GraphDataScience(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# Print GDS version to confirm connection
print(f"GDS Version: {gds.version()}")


SecretNotFoundError: Secret secretName does not exist.

In [17]:
# Count the number of nodes in the database
nodes = gds.run_cypher("MATCH (n) RETURN COUNT(n) AS node_count")
print(f"Number of nodes: {nodes}")


NameError: name 'gds' is not defined

loading dataset intopandas data frame  Load MovieLens data
i have used data from Grouplens.org 1Mdataset


In [10]:
movies=pd.read_csv("/content/drive/MyDrive/movies.dat",sep="::",encoding="ISO-8859-1",names=["MovieID","Title","Genres"])
ratings=pd.read_csv("/content/drive/MyDrive/ratings.dat",sep="::",encoding="ISO-8859-1",names=["UserID","MovieID","Rating","Timestamp"])
users=pd.read_csv("/content/drive/MyDrive/users.dat",sep="::",encoding="ISO-8859-1",names=["UserID","Gender","Age","Occupation","Zip-code"])


<ipython-input-10-02dc48f52e28>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies=pd.read_csv("/content/drive/MyDrive/movies.dat",sep="::",encoding="ISO-8859-1",names=["MovieID","Title","Genres"])
<ipython-input-10-02dc48f52e28>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings=pd.read_csv("/content/drive/MyDrive/ratings.dat",sep="::",encoding="ISO-8859-1",names=["UserID","MovieID","Rating","Timestamp"])
<ipython-input-10-02dc48f52e28>:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' 

pre reviewing the data

In [18]:
display(movies.head())
display(ratings.head())
display(users.head())

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


**bold text**

create user nodes have unique id and import user data to neo4j

In [14]:
# Create a uniqueness constraint on the User nodes
gds.run_cypher('CREATE CONSTRAINT IF NOT EXISTS ON (n:User) ASSERT n.id IS NOT NULL')

# Create User nodes from the users dataset
create_users_result = gds.run_cypher(
    '''
    UNWIND $data AS row
    MERGE (n:User {id: row.UserID})
    SET n.Gender = row.Gender, n.Age = row.Age
    RETURN COUNT(*) AS users_created
    ''',
    params={'data': users.to_dict('records')}
)
print(create_users_result)


NameError: name 'gds' is not defined

create movie nodes and ensure unique id and setting Title and Genere

In [15]:
# Create a uniqueness constraint on the Movie nodes
gds.run_cypher('CREATE CONSTRAINT IF NOT EXISTS ON (n:Movie) ASSERT n.id IS NOT NULL')

# Create Movie nodes from the movies dataset
create_movies_result = gds.run_cypher(
    '''
    UNWIND $data AS row
    MERGE (n:Movie {id: row.MovieID})
    SET n.Title = row.Title, n.Genres = row.Genres
    RETURN COUNT(*) AS movies_created
    ''',
    params={'data': movies.to_dict('records')}
)
print(create_movies_result)

NameError: name 'gds' is not defined

 Create Relationships (User -> Rated -> Movie)
 chunk processing split data frame into smaller chunks to handle large dataset

 unwind to process each row chunk as a parmeter
 create relationship between User and Movie to store rating relationship


In [19]:
# Create RATED relationships in batches to handle large datasets
i = 1
for chunk in np.array_split(ratings, 200):
    if i % 10 == 0:
        print(f"Processing batch {i}")
    gds.run_cypher(
        '''
        UNWIND $data AS row
        MATCH (u:User {id: row.UserID}), (m:Movie {id: row.MovieID})
        MERGE (u)-[r:RATED]->(m)
        SET r.Rating = row.Rating
        ''',
        params={'data': chunk.to_dict('records')}
    )
    i += 1

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


NameError: name 'gds' is not defined

to visualize the schema use


In [ ]:
# Check the schema
schema = gds.run_cypher("CALL apoc.meta.schema()")
print(schema)


In [ ]:
# Visualize the schema
visualization = gds.run_cypher("CALL db.schema.visualization()")
print(visualization)

now i have found similar movie by identfying rated highly by the same user who liked the specifiyed movie
then filters and scoring  filter movie that are not in target movie and counts common users for ranking

In [ ]:

similar_movies_toy_story = gds.run_cypher(
    '''
    MATCH (m1:Movie)-[r1:RATED]-(u:User)-[r2:RATED]-(m2:Movie)
    WHERE m1.Title CONTAINS 'Toy Story (1995)'
      AND m2.Title <> 'Toy Story (1995)'
      AND r1.Rating = 5 AND r2.Rating = 5
    RETURN m2.Title AS Movie, m2.Genres AS Genres, COUNT(DISTINCT u) AS CommonUsers
    ORDER BY CommonUsers DESC
    '''
)
print(similar_movies_toy_story.head())

graph projection projects the data into graph to calculate similarity calculation and graph algorthim

Using node similarity Given two sets A and B the jasccard similarityis computed using j(A,B)=|A n B|/|A U B|

Jaccard similariy for node1 and node 2 above is J({e1,e2},{e1})=|{e1,e2} n {e1}|/|{e1,e2}U{e1}| =|{e1}|/{e1,e2}=1/2=0.5
User Based recommendation i have created recommendation for a user this id done first calculatiing a amilarity measure /score between users for this i use jaccard similarity

1 first create projection 2 calculate actual jaccard similarity

In [ ]:
#  graph projection for GDS algorithms
create_projection = gds.run_cypher(
    '''
    CALL gds.graph.project(
      'myGraph',
      ['User', 'Movie'],
      {
        RATED: {properties: 'Rating'}
      }
    )
    '''
)
print(create_projection)

In [ ]:
# List available graphs
graphs = gds.run_cypher(
    '''
    CALL gds.graph.list()
    YIELD graphName, nodeCount, relationshipCount
    RETURN graphName, nodeCount, relationshipCount
    ORDER BY graphName ASC
    '''
)
print(graphs)


Node Similarity Algorithm: Calculates similarity scores between users based on their ratings.
Streaming Results: Returns a ranked list of user pairs with similarity scores.

In [ ]:
user_similarity = gds.run_cypher(
    '''
    CALL gds.nodeSimilarity.stream('myGraph')
    YIELD node1, node2, similarity
    RETURN gds.util.asNode(node1).id AS UserID1, gds.util.asNode(node2).id AS UserID2, similarity
    ORDER BY similarity DESC, UserID1, UserID2
    '''
)
print(user_similarity.head())

Movies for user retrives movies rated by specfic user and sorts then by rating in descinding order


In [ ]:
# Get movie recommendations for a specific user
movies_for_user = gds.run_cypher(
    '''
    MATCH (u:User)-[r:RATED]->(m:Movie)
    WHERE id(u) = $id
    RETURN m.Title AS Title, m.Genres AS Genres, r.Rating AS Rating
    ORDER BY r.Rating DESC
    ''',
    params={'id': 4725}
)
print(movies_for_user.head(10))


creating similar realionship  establish SIMILAR realionships between users based on there similarity scores

In [ ]:
# create SIMILAR relationships based on user similarity
i = 1
for chunk in np.array_split(user_similarity.query('UserID1 > UserID2'), 10):
    print(f"Processing similarity batch {i}")
    create_similar = gds.run_cypher(
        '''
        UNWIND $data AS row
        MATCH (u1:User {id: row.UserID1}), (u2:User {id: row.UserID2})
        MERGE (u1)-[r:SIMILAR]->(u2)
        SET r.similarity = row.similarity
        RETURN COUNT(*) AS relationships_created
        ''',
        params={'data': chunk.to_dict('records')}
    )
    print(create_similar)
    i += 1


 Recommended Movies Based on similarity

i have calculted recommedation score for each movie based on the similariy of users and their ratings excluding movies the user has already rated

In [ ]:
# Recommend movies to a user based on similarity scores
similar_movies_for_user = gds.run_cypher(
    '''
    MATCH (u1:User)-[r:SIMILAR]-(u2)-[r2:RATED]->(m:Movie)
    WHERE id(u1) = $id AND NOT EXISTS {
        MATCH (u1)-[:RATED]->(m)
    }
    RETURN m.Title AS Title, m.Genres AS Genres,
           SUM(r.similarity * r2.Rating) / (SUM(r.similarity) + LOG(COUNT(r2))) AS Score
    ORDER BY Score DESC
    ''',
    params={'id': 4725}
)
print(similar_movies_for_user.head(10))